In [1]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [2]:
%reload_kedro

2022-08-16 14:15:52,557 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-16 14:15:52,604 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-16 14:15:53,147 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-16 14:15:53,147 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-16 14:15:53,157 - root - INFO - Registered line magic `run_viz`
2022-08-16 14:15:53,157 - root - INFO - Registered line magic `reload_kedro_mlflow`


In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [5]:
catalog.list()

['articles',
 'customers',
 'transactions',
 'customers_sample',
 'transactions_sample',
 'articles_sample',
 'input_images',
 'resized_images',
 'image_embeddings',
 'text_embeddings',
 'candidates_sample',
 'automated_articles_features',
 'automated_customers_features',
 'manual_article_features',
 'manual_customer_features',
 'manual_customer_prod_group_features',
 'jaccard_similarity_features',
 'image_cosine_similarity_features',
 'text_cosine_similarity_features',
 'parameters',
 'params:automated_feature_engineering',
 'params:automated_feature_engineering.articles',
 'params:automated_feature_engineering.articles.feature_selection',
 'params:automated_feature_engineering.articles.feature_selection_params',
 'params:automated_feature_engineering.articles.feature_selection_params.pct_null_threshold',
 'params:automated_feature_engineering.articles.feature_selection_params.corr_threshold',
 'params:automated_feature_engineering.customers',
 'params:automated_feature_engineering.cu

In [6]:
candidates = context.catalog.load('candidates_sample')

2022-08-16 14:15:54,076 - kedro.io.data_catalog - INFO - Loading data from `candidates_sample` (ParquetDataSet)...


In [7]:
candidates.shape

(137198, 7)

In [8]:
candidates.isna().sum()/candidates.shape[0]

customer_id                   0.0000
global_articles               0.0000
segment_articles              0.0000
previously_bought             0.6418
previously_bought_prod_name   0.6418
closest_image_embeddings      0.6421
closest_text_embeddings       0.6418
dtype: float64

In [9]:
candidates.set_index('customer_id', inplace=True)

In [10]:
dfs = list()

In [11]:
%%time
for column in candidates.columns:
    print(column)
    df = candidates[[column]].explode(column).rename({column: 'article_id'}, axis=1).dropna()
    df = df.assign(strategy_name=lambda x: column)
    print(df.shape)
    dfs.append(df)

global_articles
(33339114, 2)
segment_articles
(34022483, 2)
previously_bought
(137416, 2)
previously_bought_prod_name
(297283, 2)
closest_image_embeddings
(451864, 2)
closest_text_embeddings
(478981, 2)
CPU times: user 8.13 s, sys: 1.02 s, total: 9.14 s
Wall time: 9.23 s


In [12]:
dfs[4]

,article_id,strategy_name
customer_id,,
64965f0125720360bbb29e1ace87acd0f483703d732d26e8ee4d35cf379fa14e,0905507002,closest_image_embeddings
64965f0125720360bbb29e1ace87acd0f483703d732d26e8ee4d35cf379fa14e,0701084002,closest_image_embeddings
64965f0125720360bbb29e1ace87acd0f483703d732d26e8ee4d35cf379fa14e,0713875002,closest_image_embeddings
64965f0125720360bbb29e1ace87acd0f483703d732d26e8ee4d35cf379fa14e,0757740006,closest_image_embeddings
64965f0125720360bbb29e1ace87acd0f483703d732d26e8ee4d35cf379fa14e,0788580001,closest_image_embeddings
...,...,...
a4092d95d91ec0ff90527962935550346d993a299353c6cf3d00d9fc7b9cc74b,0737948002,closest_image_embeddings
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,0650388001,closest_image_embeddings
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,0711138001,closest_image_embeddings


In [13]:
candidates = pd.concat(dfs, axis=0).reset_index()

In [14]:
candidates.shape

(68727141, 3)

In [15]:
candidates.head()

,customer_id,article_id,strategy_name
0,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0554479001,global_articles
1,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0904571001,global_articles
2,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0869331006,global_articles
3,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0792469001,global_articles
4,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0902528006,global_articles


#### other dataframes

In [16]:
articles = context.catalog.load('articles')

2022-08-16 14:16:15,310 - kedro.io.data_catalog - INFO - Loading data from `articles` (CSVDataSet)...


In [17]:
customers = context.catalog.load('customers_sample')

2022-08-16 14:16:15,639 - kedro.io.data_catalog - INFO - Loading data from `customers_sample` (CSVDataSet)...


In [18]:
transactions = context.catalog.load('transactions_sample')

2022-08-16 14:16:15,854 - kedro.io.data_catalog - INFO - Loading data from `transactions_sample` (CSVDataSet)...


In [19]:
articles.shape, customers.shape, transactions.shape

((105542, 25), (137198, 7), (157785, 5))

### candidate features

##### jaccard similarity

The actual comparison was actually simpler. I calculated average jaccard index of attributes. So when a customer had 3 items in the history:

item_1 attributes A,B,C  
item_2 attributes B,C,D  
item_3 attributes A,B,D

Then you consider a new item_4 with attributes A,B so the average jaccard is calculated like this

item_1 attributes A,B,C = jaccard 0.66  
item_2 attributes B,C,D = jaccard 0.33  
item_3 attributes A,B,D = jaccard 0.66

so the average similarity is 0.55

I had a different version to calculate "fuzzy" attribute similarity based on their cooccurence in baskets so if the attribute=X cooccured with attribute=Y in 50% of the baskets its similarity is 50% not 0 or 1 like in the original.

In [20]:
articles.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [21]:
cat_articles = articles[['product_type_name', 'product_group_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'index_name', 'index_group_name', 'section_name', 'garment_group_name']]

In [22]:
jaccard_articles = articles[['article_id']].copy()

In [23]:
jaccard_articles['set_of_attributes'] = cat_articles.apply(set, axis=1)

In [24]:
jaccard_articles

,article_id,set_of_attributes
0,0108775015,"{Garment Upper body, Vest top, Black, Solid, W..."
1,0108775044,"{Garment Upper body, Vest top, Light, Solid, W..."
2,0108775051,"{Garment Upper body, Vest top, Dusty Light, Wo..."
3,0110065001,"{Bra, Black, Solid, Underwear, Lingeries/Tight..."
4,0110065002,"{Light, Bra, Solid, Underwear, White, Lingerie..."
...,...,...
105537,0953450001,"{Men Underwear, Socks & Tights, Socks and Tigh..."
105538,0953763001,"{Garment Upper body, Vest top, H&M+, Black, So..."
105539,0956217002,"{Garment Full body, Womens Trend, Black, Solid..."
105540,0957375001,"{Divided, Hair clip, Divided Accessories, Acce..."


In [25]:
jaccard_articles.set_index(['article_id'], inplace=True)

In [26]:
jaccard_articles

,set_of_attributes
article_id,
0108775015,"{Garment Upper body, Vest top, Black, Solid, W..."
0108775044,"{Garment Upper body, Vest top, Light, Solid, W..."
0108775051,"{Garment Upper body, Vest top, Dusty Light, Wo..."
0110065001,"{Bra, Black, Solid, Underwear, Lingeries/Tight..."
0110065002,"{Light, Bra, Solid, Underwear, White, Lingerie..."
...,...
0953450001,"{Men Underwear, Socks & Tights, Socks and Tigh..."
0953763001,"{Garment Upper body, Vest top, H&M+, Black, So..."
0956217002,"{Garment Full body, Womens Trend, Black, Solid..."


### works

In [102]:
def jaccard(x, y):
    """ returns the jaccard similarity between two sets"""
    intersection_cardinality = len(x.intersection(y))
    union_cardinality = len(x.union(y))
    return float(intersection_cardinality/union_cardinality)

In [111]:
df_cust_article = transactions[['customer_id', 'article_id']].drop_duplicates().groupby(['customer_id'])['article_id'].apply(list).reset_index().set_index('customer_id')

In [112]:
df_cust_article.head()

,article_id
customer_id,
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,[0351484002]
0000b2f1829e23b24feec422ef13df3ccedaedc85368e6664d04ca30b2f8daff,"[0715411001, 0706016015]"
000346516dd355b40badca0c0f5f37a318ddae31f0e0f76a3a0454eb591b6384,"[0816588001, 0854193002, 0816423002]"
000531ecd2754a0cf214373cc7b6dbcf7ce9a77c3284a2facbe072f763618bbc,[0688105016]
000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060ccf17ba290e00c5d85,"[0488561003, 0570002002, 0716545008]"


In [114]:
df_cust_article.loc['000346516dd355b40badca0c0f5f37a318ddae31f0e0f76a3a0454eb591b6384'][0]

['0816588001', '0854193002', '0816423002']

In [126]:
def calculate_avg_jaccard_similarity(
    candidates_df: pd.DataFrame,
    articles_attributes: pd.DataFrame,
    df_cust_article: pd.DataFrame):
    
    candidate_item, candidate_user = candidates_df.article_id, candidates_df.customer_id
    candidate_item_attributes = articles_attributes.loc[candidate_item]['set_of_attributes']
    try:
        bought_items = df_cust_article.loc[candidate_user][0]
    except KeyError:
        return 0
    jaccard_similarity_list = []
    for item in bought_items:
        item_attributes = articles_attributes.loc[item]['set_of_attributes']
        jaccard_similarity = jaccard(candidate_item_attributes, item_attributes)
        jaccard_similarity_list.append(jaccard_similarity)
    return sum(jaccard_similarity_list)/len(jaccard_similarity_list)    

In [127]:
candidates.head()

,customer_id,article_id,strategy_name
0,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0554479001,global_articles
1,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0904571001,global_articles
2,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0869331006,global_articles
3,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0792469001,global_articles
4,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,0902528006,global_articles


In [128]:
sample_candidates = candidates.sample(100_000)

In [129]:
sample_candidates.shape

(100000, 3)

### apply

In [134]:
%%time
sample_candidates.apply(lambda x: calculate_avg_jaccard_similarity(x, jaccard_articles, df_cust_article), axis=1)

CPU times: user 6.62 s, sys: 29.3 ms, total: 6.65 s
Wall time: 6.65 s


38424402   0.0000
41710740   0.0000
6535910    0.0000
41639791   0.0667
42162267   0.0778
            ...  
51526204   0.0000
64710650   0.0000
63483160   0.0000
50661274   0.0000
32157985   0.0000
Length: 100000, dtype: float64

In [34]:
9*candidates.shape[0]/sample_candidates.shape[0]/60

103.0907115

In [133]:
(9*60)/6.59

81.94233687405159

after

In [137]:
6.59*candidates.shape[0]/sample_candidates.shape[0]/60/60

1.25808849775

### itertuples

In [135]:
%%time
pd.Series(
    calculate_avg_jaccard_similarity(row, jaccard_articles, df_cust_article)
    for row in sample_candidates.itertuples()
)

CPU times: user 5.61 s, sys: 14.9 ms, total: 5.62 s
Wall time: 5.62 s


0       0.0000
1       0.0000
2       0.0000
3       0.0667
4       0.0778
         ...  
99995   0.0000
99996   0.0000
99997   0.0000
99998   0.0000
99999   0.0000
Length: 100000, dtype: float64

In [34]:
9*candidates.shape[0]/sample_candidates.shape[0]/60

103.0907115

### maybe faster jaccard_similarity

In [27]:
def jaccard(x, y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [85]:
def jaccard2(x, y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(x.intersection(y))
    union_cardinality = len(x.union(y))
    return intersection_cardinality/float(union_cardinality)

In [92]:
def jaccard3(x, y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(x.intersection(y))
    union_cardinality = len(x.union(y))
    return float(intersection_cardinality/union_cardinality)

In [77]:
a = jaccard_articles.sample(1).set_of_attributes[0]

In [78]:
b = jaccard_articles.sample(1).set_of_attributes[0]

In [90]:
%timeit -n 5000 jaccard(a, b)

1.01 µs ± 20 ns per loop (mean ± std. dev. of 7 runs, 5000 loops each)


In [100]:
%timeit -n 5000 jaccard2(a, b)

457 ns ± 34.8 ns per loop (mean ± std. dev. of 7 runs, 5000 loops each)


In [101]:
%timeit -n 5000 jaccard3(a, b)

443 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 5000 loops each)


### image/text embeddings

In [20]:
image_embeddings = context.catalog.load('image_embeddings')

2022-08-16 14:16:15,987 - kedro.io.data_catalog - INFO - Loading data from `image_embeddings` (ParquetDataSet)...


In [21]:
image_embeddings

,img_emb_1,img_emb_2,img_emb_3,img_emb_4,img_emb_5,img_emb_6,img_emb_7,img_emb_8,img_emb_9,img_emb_10,...,img_emb_23,img_emb_24,img_emb_25,img_emb_26,img_emb_27,img_emb_28,img_emb_29,img_emb_30,img_emb_31,img_emb_32
0520567001,1.1136,1.6530,-2.1507,2.0899,1.1859,2.2600,0.3472,2.0131,-4.7345,2.4018,...,2.0007,-0.2031,-2.8428,0.4911,-0.0419,2.5446,0.1087,1.5822,4.2004,-2.0204
0878510001,1.4409,1.3867,-2.2826,2.1407,0.8620,2.1601,0.0811,1.7678,-4.8009,1.9922,...,1.6910,-0.1554,-3.0912,0.5878,-0.4512,2.7649,0.4721,1.5510,4.1647,-2.0769
0573864007,-0.7568,1.5158,-0.5756,0.6109,1.7299,1.2196,1.0663,1.6654,-1.9784,2.5785,...,1.9107,-0.5298,-0.3629,-0.1672,1.4940,0.2434,-1.1655,1.0513,2.0177,-0.7936
0832893002,1.1667,1.3586,-2.2046,1.9648,1.0202,2.0245,0.1760,1.7535,-4.6813,2.1408,...,1.7510,-0.3855,-2.8435,0.4786,-0.1807,2.4874,0.3440,1.6696,4.0890,-2.0683
0654410024,0.9548,1.6560,-1.9917,1.9652,1.2261,2.1809,0.4167,1.9897,-4.4603,2.4084,...,1.9925,-0.1926,-2.6166,0.4386,0.0854,2.3450,-0.0179,1.5020,3.9832,-1.8879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0633529002,1.3275,1.4195,-2.1956,2.0770,0.9209,2.1351,0.1450,1.7867,-4.6704,2.0465,...,1.7254,-0.1601,-2.9580,0.5514,-0.3419,2.6463,0.3709,1.5213,4.0722,-2.0117
0822895002,0.4658,1.8164,-1.5941,1.6946,1.4782,2.0865,0.7031,2.0820,-3.8644,2.6402,...,2.1455,-0.1754,-2.0297,0.2855,0.5559,1.8323,-0.4750,1.3414,3.5597,-1.5813
0759602003,0.2192,1.4499,-1.8595,1.4416,1.6240,1.7125,0.6022,1.8542,-4.2194,2.7570,...,2.0814,-0.9784,-1.9779,0.1264,0.7923,1.5771,-0.2489,1.9445,3.8230,-1.9520
0808866004,1.3212,1.3889,-2.2574,2.0748,0.9433,2.1152,0.1294,1.7773,-4.7725,2.0773,...,1.7342,-0.2571,-2.9936,0.5410,-0.3257,2.6536,0.4098,1.6110,4.1551,-2.0828


In [23]:
df_cust_article = transactions[['customer_id', 'article_id']].drop_duplicates().groupby(['customer_id'])['article_id'].apply(list).reset_index().set_index('customer_id')

In [24]:
df_cust_article.head()

,article_id
customer_id,
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,[0351484002]
0000b2f1829e23b24feec422ef13df3ccedaedc85368e6664d04ca30b2f8daff,"[0715411001, 0706016015]"
000346516dd355b40badca0c0f5f37a318ddae31f0e0f76a3a0454eb591b6384,"[0816588001, 0854193002, 0816423002]"
000531ecd2754a0cf214373cc7b6dbcf7ce9a77c3284a2facbe072f763618bbc,[0688105016]
000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060ccf17ba290e00c5d85,"[0488561003, 0570002002, 0716545008]"


In [25]:
def calculate_customer_embedding(df: pd.DataFrame, embeddings: pd.DataFrame):
    list_of_articles = df.article_id[0]
    mean_embeddings = list(embeddings[embeddings.index.isin(list_of_articles)].mean(axis=0))
    return mean_embeddings

In [26]:
%%time
cust_embeddings = df_cust_article.groupby(['customer_id']).apply(lambda x: calculate_customer_embedding(x, image_embeddings)).reset_index(name='embeddings')

CPU times: user 2min 4s, sys: 312 ms, total: 2min 4s
Wall time: 2min 4s


,customer_id,embeddings
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0.29205620288848877, 1.5997645854949951, -1.8..."
1,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,"[-0.3882942795753479, 2.040398120880127, -1.24..."
2,000346516dd355b40badca0c0f5f37a318ddae31f0e0f7...,"[0.35325416922569275, 1.3315706253051758, -1.9..."
3,000531ecd2754a0cf214373cc7b6dbcf7ce9a77c3284a2...,"[0.435394287109375, 1.4381957054138184, -1.931..."
4,000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060...,"[0.7979548573493958, 1.5542672872543335, -1.89..."
...,...,...
49140,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,"[0.7364576458930969, 1.5763212442398071, -1.94..."
49141,fff69e3ab1bd701315881c7706c38d5cbcda1f4e0cf213...,"[-0.35780400037765503, 1.5779448747634888, -1...."
49142,fff94b7e15cd61eaf699f0e7d29e4023d28733119f323d...,"[-0.7281959056854248, 1.4384605884552002, -1.4..."
49143,fffdd9c81f64c0228a54e1c0240cf40877acec6ce985d4...,"[1.3642925024032593, 1.3897253274917603, -2.20..."


In [27]:
def calculate_customer_embedding(df: pd.DataFrame, embeddings: pd.DataFrame):
    list_of_articles = list(df.article_id.unique())
    mean_embeddings = list(embeddings[embeddings.index.isin(list_of_articles)].mean(axis=0))
    return mean_embeddings

In [28]:
%%time
cust_embeddings = transactions.groupby(['customer_id']).apply(lambda x: calculate_customer_embedding(x, image_embeddings)).reset_index(name='embeddings')

CPU times: user 2min 4s, sys: 292 ms, total: 2min 5s
Wall time: 2min 5s


,customer_id,embeddings
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0.29205620288848877, 1.5997645854949951, -1.8..."
1,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,"[-0.3882942795753479, 2.040398120880127, -1.24..."
2,000346516dd355b40badca0c0f5f37a318ddae31f0e0f7...,"[0.35325416922569275, 1.3315706253051758, -1.9..."
3,000531ecd2754a0cf214373cc7b6dbcf7ce9a77c3284a2...,"[0.435394287109375, 1.4381957054138184, -1.931..."
4,000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060...,"[0.7979548573493958, 1.5542672872543335, -1.89..."
...,...,...
49140,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,"[0.7364576458930969, 1.5763212442398071, -1.94..."
49141,fff69e3ab1bd701315881c7706c38d5cbcda1f4e0cf213...,"[-0.35780400037765503, 1.5779448747634888, -1...."
49142,fff94b7e15cd61eaf699f0e7d29e4023d28733119f323d...,"[-0.7281959056854248, 1.4384605884552002, -1.4..."
49143,fffdd9c81f64c0228a54e1c0240cf40877acec6ce985d4...,"[1.3642925024032593, 1.3897253274917603, -2.20..."


In [184]:
cust_embeddings.shape

(49145, 2)

In [185]:
image_embeddings.shape

(105100, 32)

### text

In [29]:
text_embeddings = context.catalog.load('text_embeddings')

2022-08-16 14:27:47,878 - kedro.io.data_catalog - INFO - Loading data from `text_embeddings` (ParquetDataSet)...


In [30]:
text_embeddings.shape

(105542, 384)

In [31]:
def calculate_customer_embedding(df: pd.DataFrame, embeddings: pd.DataFrame):
    list_of_articles = df.article_id[0]
    mean_embeddings = list(embeddings[embeddings.index.isin(list_of_articles)].mean(axis=0))
    return mean_embeddings

In [32]:
%%time
cust_embeddings = df_cust_article.groupby(['customer_id']).apply(lambda x: calculate_customer_embedding(x, text_embeddings)).reset_index(name='embeddings')

CPU times: user 2min 3s, sys: 719 ms, total: 2min 4s
Wall time: 2min 4s


In [33]:
def calculate_customer_embedding(df: pd.DataFrame, embeddings: pd.DataFrame):
    list_of_articles = list(df.article_id.unique())
    mean_embeddings = list(embeddings[embeddings.index.isin(list_of_articles)].mean(axis=0))
    return mean_embeddings

In [34]:
%%time
cust_embeddings = transactions.groupby(['customer_id']).apply(lambda x: calculate_customer_embedding(x, text_embeddings)).reset_index(name='embeddings')

CPU times: user 2min 4s, sys: 805 ms, total: 2min 5s
Wall time: 2min 5s


In [137]:
%%time
transactions.groupby(['customer_id']).apply(lambda x: calculate_customer_embedding(x, text_embeddings)).reset_index(name='embeddings')

CPU times: user 1min 56s, sys: 702 ms, total: 1min 56s
Wall time: 1min 56s


,customer_id,embeddings
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0.060339782387018204, 0.017903417348861694, 0..."
1,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,"[-0.03108256496489048, 0.07063155621290207, -0..."
2,000346516dd355b40badca0c0f5f37a318ddae31f0e0f7...,"[-0.03189806267619133, 0.08947073668241501, -0..."
3,000531ecd2754a0cf214373cc7b6dbcf7ce9a77c3284a2...,"[-0.007199143059551716, 0.10448223352432251, 0..."
4,000989f72a2b8e5da2f4abafc86c2e213816fa2ff2a060...,"[-0.02177959680557251, 0.09326094388961792, 0...."
...,...,...
49140,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,"[-0.04204563423991203, 0.05189591646194458, 0...."
49141,fff69e3ab1bd701315881c7706c38d5cbcda1f4e0cf213...,"[0.07588859647512436, 0.014239171519875526, -0..."
49142,fff94b7e15cd61eaf699f0e7d29e4023d28733119f323d...,"[0.04923105612397194, 0.0729655772447586, 0.03..."
49143,fffdd9c81f64c0228a54e1c0240cf40877acec6ce985d4...,"[-0.049292903393507004, 0.0693088248372078, 0...."


to split embeddings from list to single columns

df3 = df2.teams.apply(pd.Series) <br>
df3.columns = ['team1', 'team2']

In [198]:
np.array(cust_embeddings.set_index('customer_id').loc['000531ecd2754a0cf214373cc7b6dbcf7ce9a77c3284a2facbe072f763618bbc'].embeddings)

array([ 0.43539429,  1.43819571, -1.93127227,  1.56413281,  1.48725712,
        1.79077113,  0.50990307,  1.83755374, -4.31625366,  2.61858845,
        0.31816185,  1.70420766, -2.99186277,  1.39999628, -0.2818093 ,
       -0.97360808, -0.16821225, -0.3271288 , -2.23177528,  6.11882687,
        0.79731834,  2.77756238,  2.01025057, -0.82952678, -2.17244005,
        0.2082431 ,  0.57116681,  1.78582966, -0.12209548,  1.87108564,
        3.87817287, -1.97070253])

In [194]:
image_embeddings.loc['0878510001'].values

array([ 1.4408963e+00,  1.3866708e+00, -2.2826171e+00,  2.1407368e+00,
        8.6195004e-01,  2.1600864e+00,  8.1075065e-02,  1.7677919e+00,
       -4.8008981e+00,  1.9921565e+00,  1.2470684e+00,  1.4888606e+00,
       -2.9717569e+00,  5.6244093e-01, -1.0710030e+00, -1.5912099e+00,
        6.6585618e-01,  6.0829879e-03, -1.7968456e+00,  6.5001235e+00,
        9.7213534e-04,  2.9396386e+00,  1.6909531e+00, -1.5535519e-01,
       -3.0912085e+00,  5.8781457e-01, -4.5122060e-01,  2.7648869e+00,
        4.7212821e-01,  1.5510160e+00,  4.1646776e+00, -2.0769293e+00],
      dtype=float32)

In [230]:
def cosine_similarity(A, B):
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [235]:
def cosine_embedding_similarity(candidates_df, customers_embeddings, items_embeddings):
    """
    """
    candidate_item, candidate_user = candidates_df.article_id, candidates_df.customer_id
    try:
        customer_embeddings = np.array(cust_embeddings.set_index('customer_id').loc[candidate_user].embeddings)
    except KeyError:
        return 0
    try:
        candidate_embeddings = items_embeddings.loc[candidate_item].values
    except KeyError:
        return 0
    return cosine_similarity(customer_embeddings, candidate_embeddings)

In [239]:
candidates_df = pd.DataFrame({'customer_id': sorted(customers['customer_id'].sample(5).to_list()*3),
                              'article_id': articles['article_id'].sample(15).to_list()
                             })

In [240]:
candidates_df.head()

,customer_id,article_id
0,43be7a90d1b3b4ddc4486a2b9ca4d8dc69d73791e99cb7...,0584198001
1,43be7a90d1b3b4ddc4486a2b9ca4d8dc69d73791e99cb7...,0863587003
2,43be7a90d1b3b4ddc4486a2b9ca4d8dc69d73791e99cb7...,0695545008
3,a60d2b71f2728406f419fb65e761435c5338554170f3cb...,0539816001
4,a60d2b71f2728406f419fb65e761435c5338554170f3cb...,0841874001


In [241]:
candidates_df.apply(lambda x: cosine_embedding_similarity(x, cust_embeddings, image_embeddings), axis=1)

0    0.0000
1    0.0000
2    0.0000
3    0.9625
4    0.8308
5    0.9894
6    0.0000
7    0.0000
8    0.0000
9    0.0000
10   0.0000
11   0.0000
12   0.0000
13   0.0000
14   0.0000
dtype: float64

text similarities

In [242]:
%%time
cust_embeddings = transactions.groupby(['customer_id']).apply(lambda x: calculate_customer_embedding(x, text_embeddings)).reset_index(name='embeddings')

CPU times: user 1min 56s, sys: 707 ms, total: 1min 57s
Wall time: 1min 57s


In [243]:
candidates_df.apply(lambda x: cosine_embedding_similarity(x, cust_embeddings, text_embeddings), axis=1)

0    0.0000
1    0.0000
2    0.0000
3    0.4155
4    0.3849
5    0.4410
6    0.0000
7    0.0000
8    0.0000
9    0.0000
10   0.0000
11   0.0000
12   0.0000
13   0.0000
14   0.0000
dtype: float64

all similarities

collaborative filtering still AWAITS :<<<

candidates -> STRATEGY FEATURE, MAYBE SCORE?